In [1]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
le = LabelEncoder()

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)
test_keep = test
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
crime_train, crime_test = train_test_split(crime_in_sf, test_size=.4)

In [3]:
for column in test.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(test[column])
        test[column] = le.transform(test[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])
        crime_train[column] = le.transform(crime_train[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])        
        crime_test[column] = le.transform(crime_test[column])

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
categories = crime_train.Category
crime_train = crime_train.drop('Category', axis=1)

categories2 = crime_test.Category
crime_test = crime_test.drop('Category', axis=1)

In [5]:
dtrain = xgb.DMatrix(crime_train.as_matrix(),
                     label=categories)
dtest = xgb.DMatrix(crime_test.as_matrix(),
                    label=categories2)

In [6]:
param = {'bst:max_depth':6, 'objective':'multi:softprob', 'num_class':39}
param['nthread'] = 4
param['eval_metric'] = ['merror', 'mlogloss']

In [7]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
num_round = 1000
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=5)

Will train until train error hasn't decreased in 5 rounds.
Multiple eval metrics have been passed: 'mlogloss' will be used for early stopping.

[0]	eval-merror:0.745006	eval-mlogloss:3.104488	train-merror:0.743122	train-mlogloss:3.098524
[1]	eval-merror:0.738489	eval-mlogloss:2.926242	train-merror:0.736664	train-mlogloss:2.917984
[2]	eval-merror:0.736473	eval-mlogloss:2.810864	train-merror:0.733982	train-mlogloss:2.800483
[3]	eval-merror:0.734878	eval-mlogloss:2.729102	train-merror:0.731854	train-mlogloss:2.716387
[4]	eval-merror:0.734209	eval-mlogloss:2.667837	train-merror:0.730801	train-mlogloss:2.653054
[5]	eval-merror:0.733082	eval-mlogloss:2.620635	train-merror:0.729337	train-mlogloss:2.603992
[6]	eval-merror:0.732646	eval-mlogloss:2.583580	train-merror:0.728665	train-mlogloss:2.564946
[7]	eval-merror:0.731524	eval-mlogloss:2.553233	train-merror:0.727344	train-mlogloss:2.532571
[8]	eval-merror:0.730961	eval-mlogloss:2.529043	train-merror:0.726424	train-mlogloss:2.506683
[9]	eval-m

In [ ]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [ ]:
predictions = pd.DataFrame(predictions)

In [ ]:
le.fit(crime_in_sf.Category)
predictions.columns = le.inverse_transform(predictions.columns)

In [ ]:
pred_test = predictions

In [ ]:
predictions['Id'] = predictions.index

In [ ]:
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame

predictions = order(predictions,['Id'])

In [ ]:
predictions.to_csv('predictions_XGB_final.csv', index=False)